In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform, randint

In [7]:
data_name = 'metadata19_hmineq0.0_tmin20050000_mean_grid_20.csv'
glathida_rgis = pd.read_csv(data_name, low_memory=False)
glathida_rgis = glathida_rgis.dropna()
df = glathida_rgis.drop(columns = 'RGIId')

In [8]:
glacierTermTypes = ('Land-terminating', 'Marine-terminating', 'Lake-terminating',
                     'Dry calving', 'Regenerated', 'Shelf-terminating', 'Not assigned')
glacierID = (0, 1, 2, 3, 4, 5, 9)

for i in range(len(glacierTermTypes)):
    print(i, glacierTermTypes[i],': ', np.size(df[df['TermType'] == glacierID[i]]))

0 Land-terminating :  3037074
1 Marine-terminating :  611040
2 Lake-terminating :  24396
3 Dry calving :  0
4 Regenerated :  0
5 Shelf-terminating :  227601
6 Not assigned :  0


Only four of the TermTypes exists. Why are the other included?

In [9]:
def split(data, test_size, target):
    # Initial data split into x and y values.
    xData = pd.DataFrame(data)
    xData = xData.drop(columns = [target])
    # Target
    yData = data[target]

    # Split into test and training data
    xTrain, xTest, yTrain, yTest = train_test_split(xData, yData, test_size = test_size, random_state = 42)

    return xTrain, xTest, yTrain, yTest

In [10]:
xTrain, xTest, yTrain, yTest = split(df, 0.2, 'TermType')

In [11]:
# Multi-layer Perceptron (MLP)
def ClassMLP_crude(xTrain, xTest, yTrain, yTest):
    # Define and train MLP classifier
    ClassifierMLP = MLPClassifier(random_state = 42, early_stopping=True)
    
    # Train the model on training data
    ClassifierMLP.fit(xTrain, yTrain)

    # Compute predictions from trained model:
    pre = ClassifierMLP.predict(xTest)
    acc = accuracy_score(yTest, pre)
    
    return acc, pre    

In [12]:
acc, pre = ClassMLP_crude(xTrain, xTest, yTrain, yTest)

print('Accuracy:')
print(acc)
print()
print('Clasification prediction:')
for i in range(len(glacierTermTypes)):
    print(i, glacierTermTypes[i], len(pre[np.where(pre == i)]))

Accuracy:
0.9364998173182316

Clasification prediction:
0 Land-terminating 10810
1 Marine-terminating 2082
2 Lake-terminating 17
3 Dry calving 0
4 Regenerated 0
5 Shelf-terminating 776
6 Not assigned 0


Hyperparameter optimization with grid search.

In [13]:
# Number of output neurons, Number of input neurons, Number of samples in training set, scaling factor in range 2-10.
def hiddenLayerSize(No, Ni, Ns, alpha): # Tries to estimate optimal hidden layer neurons.
    Nh = Ns / (alpha * (Ni + No))
    return Nh

Nh1 = int(hiddenLayerSize(4, np.shape(xTrain)[1], np.shape(xTrain)[0], 6))
Nh2 = int(hiddenLayerSize(4, np.shape(xTrain)[1], np.shape(xTrain)[0], 8))
Nh3 = int(hiddenLayerSize(4, np.shape(xTrain)[1], np.shape(xTrain)[0], 10))
print(f'Estimated hidden layer neurons: {Nh1}, {Nh2} and {Nh3}')

hParamGrid = {'activation': ['relu', 'tanh'],
               'alpha': [0.0001, 0.001, 0.01],
               'solver': ['adam', 'sgd'],
               'early_stopping': [True],
               'learning_rate': ['constant', 'adaptive'],
               'hidden_layer_sizes': [(Nh1, Nh1, int(Nh1 / 3)),
                                      (Nh2, Nh2, int(Nh2 / 3)), 
                                      (Nh3, Nh3, int(Nh3 / 3))]}

Estimated hidden layer neurons: 152, 114 and 91


In [14]:
def ClassMLPgrid(xTrain, xTest, yTrain, yTest, paramSpace, maxIts, crossValids):
    # Define the MLPClassifier.
    mlp = MLPClassifier(max_iter = maxIts, random_state = 42)

    print("Data normalizing.")
    scaler = StandardScaler()
    xTrain = scaler.fit_transform(xTrain)
    xTest = scaler.transform(xTest)
    
    # Perform GridSearchCV
    print("Grid search hyperparameter optimization.")
    grid_search = GridSearchCV(estimator = mlp, param_grid = paramSpace, cv = crossValids, n_jobs = -1)
    grid_search.fit(xTrain, yTrain)

    # Best parameters found during grid search.
    print("Best hyperparameters found:")
    best_params = grid_search.best_params_

    # Train MLPRegressor with best parameters.
    print("Training optimized model")
    best_mlp = MLPClassifier(max_iter = maxIts, **best_params)
    best_mlp.fit(xTrain, yTrain)

    print("Finished model")
    pre = best_mlp.predict(xTest)
    acc = best_mlp.accuracy_score(yTest, pre)

    return best_mlp, best_params, acc, pre

In [15]:
bestMLP, bestParams, acc, pre = ClassMLPgrid(xTrain, xTest, yTrain, yTest,hParamGrid, 20, 5)

print('Accuracy:')
print(acc)
print()
print(f'Best hyperparameters: {bestParams}')
print()
print('Clasification prediction:')
for i in range(len(glacierTermTypes)):
    print(i, glacierTermTypes[i], len(pre[np.where(pre == i)]))


Data normalizing.
Grid search hyperparameter optimization.


In [ ]:
NhMax = int(hiddenLayerSize(4, np.shape(xTrain)[1], np.shape(xTrain)[0], 6))
NhMin = int(hiddenLayerSize(4, np.shape(xTrain)[1], np.shape(xTrain)[0], 10))
print(f'Hidden layer neuron range estimation: {NhMin} and {NhMax}')

hParamSpace = {'activation': ['relu', 'tanh'],
               'alpha': uniform(0.001, 0.1),
               'solver': ['adam', 'sgd'],
               'early_stopping': [True],
               'learning_rate': ['constant', 'adaptive'],
               'hidden_layer_sizes': [(randint(NhMin, NhMax).rvs(),
                                       randint(NhMin, NhMax).rvs(), 
                                       int(0.3 * randint(NhMin, NhMax).rvs()))]}

Hidden layer neuron range estimation: 91 and 152


In [ ]:
def ClassMLPran(xTrain, xTest, yTrain, yTest, paramSpace, maxIts, crossValids):
    # Define the MLPClassifier.
    mlp = MLPClassifier(max_iter = maxIts, random_state = 42)

    print("Data normalizing.")
    scaler = StandardScaler()
    xTrain = scaler.fit_transform(xTrain)
    xTest = scaler.transform(xTest)
    
    # Perform GridSearchCV
    print("Grid search hyperparameter optimization.")
    grid_search = RandomizedSearchCV(estimator = mlp, param_distributions = paramSpace, cv = crossValids, n_jobs = -1)
    grid_search.fit(xTrain, yTrain)

    # Best parameters found during grid search.
    print("Best hyperparameters found:")
    best_params = grid_search.best_params_
    print(best_params)

    # Train MLPRegressor with best parameters.
    print("Training optimized model")
    best_mlp = MLPClassifier(max_iter = maxIts, **best_params)
    best_mlp.fit(xTrain, yTrain)

    print("Finished model")
    pre = best_mlp.predict(xTest)
    acc = best_mlp.accuracy_score(yTest, pre)

    return best_mlp, best_params, acc, pre

In [ ]:
bestMLP, bestParams, acc, pre = ClassMLPran(xTrain, xTest, yTrain, yTest, hParamSpace, 20, 5)

print('Accuracy:')
print(acc)
print()
print(f'Best hyperparameters: {bestParams}')
print()
print('Clasification prediction:')
for i in range(len(glacierTermTypes)):
    print(i, glacierTermTypes[i], len(pre[np.where(pre == i)]))

Data normalizing.
Grid search hyperparameter optimization.
